In [314]:
import ee
import datetime as dt
from funcs_gee import *
import xarray as xr
import rasterio
import rioxarray

# ----------------------------------------------------------------------

# Define the time range.
start_date = '2023-01-01'
end_date = '2023-12-31'
geometry_title='budrio' # input('Please provide title for geometry: ')

# Define your geometry, with coordinates [lon_min, lat_min, lon_max, lat_max]
print('Define area of interest. \nIf you have a GeoJSON file, copy paste.\n'+
      'If you have a KML, export to GeoJSON (plenty of free tools online).')

geoJSON = {
"type": "FeatureCollection",
"name": "merged",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "Name": "Budrio_campo_safe_half", "description": None, "tessellate": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 11.53262979564736, 44.570842547510622 ], [ 11.532328100248961, 44.570445732016537 ], [ 11.53264162483709, 44.570339694294631 ], [ 11.532950828277439, 44.570738040751841 ] ] ] } } ] }

nfeatures = len(geoJSON['features'])
coords_geojson = [geoJSON['features'][i]['geometry']['coordinates'] for i in range(nfeatures)]


# ----------------------------------------------------------------------

# Function to calculate NDVI.
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


# Initialize the Earth Engine module.
ee.Initialize()

region = ee.Geometry.MultiPolygon(coords)

# Load the Sentinel-2 SR Harmonized collection.
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(region)

proj=s2.first().select('B2').projection()
scale_mod=ee.Number(proj.nominalScale()).getInfo()
trans_mod=proj.getInfo()['transform']
crs_mod=proj.getInfo()['crs']

# Update masks/add bands over the collection.
s2_ndvi = s2.map(addNDVI)

# Get pixels in the region with all bands, lon, lat and time
ndvi = np.array(s2_ndvi.getRegion(region, scale_mod).getInfo())
ndvi[ndvi==None]=np.nan

# for img in ndvi[1:]:
#     img[3]=dt.datetime.fromtimestamp(img[3]/1000).strftime("%Y-%m-%d %H:%M")

# Build dataframe and clean timestamps
ndvi_df = pd.DataFrame(ndvi[1:], columns=ndvi[0])
ndvi_df['datetime'] = pd.to_datetime([dt.datetime.fromtimestamp(ts).strftime("%Y-%m-%d") for ts in ndvi_df['time']/1000])
drop_columns=['AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60','time','id']
ndvi_df.drop(drop_columns, axis=1, inplace=True)
ndvi_df['coords']=list(zip(ndvi_df.longitude, ndvi_df.latitude))
ndvi_df_unique=ndvi_df.groupby(by=['datetime','coords']).max()
ndvi_df_unique=ndvi_df_unique.astype('float')
fn=f'NDVI_{start_date}_{end_date}'

dates=np.unique(ndvi_df.datetime)

answ=input('Save nc file? [[y]/n]')
if answ=='y' or answ=='':
    ds.to_netcdf(fn+'.nc')

ds=ndvi_df_unique.reset_index().drop(columns=['coords']).set_index(['datetime', 'latitude', 'longitude']).to_xarray()

ds.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)

# Set the Coordinate Reference System (CRS) if it is not set
# You need to know the EPSG code of your CRS
# For example, the EPSG code for WGS84 is 'EPSG:4326'
ds.rio.write_crs("EPSG:4326", inplace=True)

daily_opt=input('Do you want daily images with all bands? [[y]/n]')
if daily_opt=='' or daily_opt=='y':
    for i, d in enumerate(ds.datetime):
        date_name=str(d.dt.strftime("%Y-%m-%d").values)
        data_daily=ds.sel(datetime=d)
        data_daily.rio.to_raster(f"./Maps/{date_name}.tif")
multi_opt=input('Do you want a single multilayer image with NDVI values only and days as bands? [[y]/n]')
if multi_opt=='' or multi_opt=='y':
    data_var=ds['NDVI']
    data_var.rio.to_raster(f"./Maps/{fn}.tif")

Define area of interest. 
If you have a GeoJSON file, copy paste.
If you have a KML, export to GeoJSON (plenty of free tools online).


Please provide title for geometry:  budrio
Save nc file? [[y]/n] 
Do you want daily images with all bands? [[y]/n] 
Do you want a single multilayer image with NDVI values only and days as bands? [[y]/n] 
